
# Profitable App Profiles for the App Store and Google Play Markets

Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

# Opening and Exploring the Data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

- A data set containing data about approximately ten thousand Android apps from Google Play. You can download the data set directly from this link.
- A data set containing data about approximately seven thousand iOS apps from the App Store. You can download the data set directly from this link.

Let's start by opening the two data sets and then continue with exploring the data.

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))

#Set up Apple data set as a list of lists. 
apple_opened_file = open('AppleStore.csv',encoding='utf8')
from csv import reader
apple_read_file = reader(apple_opened_file)
apple_apps_data = list(apple_read_file)

#Set up Google data set as a list of lists. 
google_opened_file = open('googleplaystore.csv',encoding='utf8')
from csv import reader
google_read_file = reader(google_opened_file)
google_apps_data = list(google_read_file)

#Check for any erroneous blank spaces in the rows:
for i in range(0,len(google_apps_data)):
    #If length of header row does not equal length of every other row, there will be an error.
    if int(len(google_apps_data[0])) != int(len(google_apps_data[i])):
        print(google_apps_data[i], f'Index number: {i}')
        
del google_apps_data[10473] #Delete row with errors. 

print(google_apps_data[10473]) #Check that row has been deleted. 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] Index number: 10473
['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


In [2]:
#Checking for any duplicate app names in the dataset. 
def duplicate_name_check(dataset, dataset_name):
    
    unique_names = []
    duplicate_names = []
    for i in dataset: 
        if i[0] not in unique_names:
            unique_names.append(i[0])
        else:
            duplicate_names.append(i[0])
    print(f'Number of duplicate app names for {dataset_name}: {len(duplicate_names)}')
        
duplicate_name_check(apple_apps_data, 'Apple')
duplicate_name_check(google_apps_data, 'Google')

#From the above, we can see that the dataset google_apps_data has 1181 duplicate app names which will 
#skew our results if they are left as is. The higher the number of reviews, the more recent the data should be. 
#Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and 
#remove the other entries for any given app. 


Number of duplicate app names for Apple: 0
Number of duplicate app names for Google: 1181


In [3]:
#Create a dictionary where each key is a unique app name and the corresponding dictionary value 
#is the highest number of reviews of that app.

reviews_max = {}
for i in google_apps_data[1:]:
    name = i[0]
    n_reviews = float(i[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max: 
        reviews_max[name] = n_reviews

print(len(reviews_max))

9659


In [4]:
#Use the dictionary created above to remove the duplicate rows.

android_clean = []
already_added = []
for i in google_apps_data[1:]:
    name = i[0]
    n_reviews = float(i[3])
    #We want to find the rows out of the duplicates and other data that have the maximum number of reviews, then 
    #add them to a new, clean list with no more duplicates. We need a separate list 
    #to keep track of the app names added to this clean list as part of the process, because otherwise
    #some of the duplicate names with the same number of reviews could be erroneously added. 
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(i)
        already_added.append(name)
        
print(len(android_clean))
        

9659


# Removing Non-English Apps

One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

Some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, to minimise the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [5]:
#We want to create a function that can filter out non-English app names from our datasets. 
def english_char(string): 
    count = 0
    for char in string: 
        if ord(char) > 127:
            count += 1
            if count > 3:
                return False
    return True

Below, we use the english_char() function to filter out the non-English apps for both data sets, then store them in new lists.

In [6]:
android_clean_english = []
apple_clean_english = []

for i in android_clean:
    name = i[0]
    if english_char(name):
        android_clean_english.append(i)
        
for i in apple_apps_data[1:]:
    name = i[2]
    if english_char(name):
        apple_clean_english.append(i)

explore_data(android_clean_english, 0, 0, True)  
explore_data(apple_clean_english, 0, 0, True)   

#We can see that we're left with 9614 Android apps and 6183 iOS apps.

Number of rows: 9614
Number of rows: 6183


# Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [15]:
#Now, we want to only include apps that are free. 
android_clean_english_free = []
apple_clean_english_free = []

for i in android_clean_english:
    price = i[7]
    if price.lower() == '0':
        android_clean_english_free.append(i)

for i in apple_clean_english:
    price = i[5]
    if price == '0':
        apple_clean_english_free.append(i)

print('Number of Android apps: ' + str(len(android_clean_english_free)), '\nNumber of Apple apps: ' + str(len(apple_clean_english_free)))
        
    
    

Number of Android apps: 8864 
Number of Apple apps: 3222


# Most Common Apps by Genre

**Part One**

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

**Part Two**

We'll build two functions we can use to analyze the frequency tables:

- One function to generate frequency tables that show percentages
- Another function that we can use to display the percentages in a descending order

In [41]:
def freq_table(dataset, index):
    dict_one = {}
    dict_one_percentages = {}
    total = 0
    for i in dataset:
        total += 1
        category = i[index]
        if category in dict_one:
            dict_one[category] += 1
        else:
            dict_one[category] = 1
    #print('\n', dict_one)
    for a in dict_one: 
        dict_one_percentages[a] = (dict_one[a]/total) * 100
        
    return dict_one_percentages
       

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


# Part Three

We start by examining the frequency table for the prime_genre column of the App Store data set.

In [43]:
print('\nprime_genre column (percentages) of the App Store data set:\n ')
display_table(apple_clean_english_free, 12)


prime_genre column (percentages) of the App Store data set:
 
Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [48]:
print('\nCategory column (percentages) of the Google Play Store data set:\n ')
display_table(android_clean_english_free, 1)


Category column (percentages) of the Google Play Store data set:
 
FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS :

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [49]:
print('\nGenres column (percentages) of the Google Play Store data set:\n ')
display_table(android_clean_english_free, 9)


Genres column (percentages) of the Google Play Store data set:
 
Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libra

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

# Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Let's start with calculating the average number of user ratings per app genre on the App Store. To do that, we'll need to:

Isolate the apps of each genre.
Sum up the user ratings for the apps of that genre.
Divide the sum by the number of apps belonging to that genre (not by the total number of apps).

In [52]:
apple_genres = freq_table(apple_clean_english_free, 12)

for genre in apple_genres:
    total = 0
    len_genre = 0
    for app in apple_clean_english_free:
        genre_app = app[12]
        if genre_app == genre:
            ratings_total = float(app[6])
            total += ratings_total
            len_genre += 1
    average_rating = total/len_genre
    print(genre, f': Average user rating is {average_rating}')
            

Productivity : Average user rating is 21028.410714285714
Weather : Average user rating is 52279.892857142855
Shopping : Average user rating is 26919.690476190477
Reference : Average user rating is 74942.11111111111
Finance : Average user rating is 31467.944444444445
Music : Average user rating is 57326.530303030304
Utilities : Average user rating is 18684.456790123455
Travel : Average user rating is 28243.8
Social Networking : Average user rating is 71548.34905660378
Sports : Average user rating is 23008.898550724636
Health & Fitness : Average user rating is 23298.015384615384
Games : Average user rating is 22788.6696905016
Food & Drink : Average user rating is 33333.92307692308
News : Average user rating is 21248.023255813954
Book : Average user rating is 39758.5
Photo & Video : Average user rating is 28441.54375
Entertainment : Average user rating is 14029.830708661417
Business : Average user rating is 7491.117647058823
Lifestyle : Average user rating is 16485.764705882353
Education 

On average, navigation apps have the highest number of user reviews.

In [61]:
for i in apple_clean_english_free:
    if i[12] == 'Navigation':
        print(i[2], '- Total ratings:', i[6])

Waze - GPS Navigation, Maps & Real-time Traffic - Total ratings: 345046
Geocaching® - Total ratings: 12811
ImmobilienScout24: Real Estate Search in Germany - Total ratings: 187
Railway Route Search - Total ratings: 5
CoPilot GPS – Car Navigation & Offline Maps - Total ratings: 3582
Google Maps - Navigation & Transit - Total ratings: 154911


Therefore, from the above, we can see that while Navigation apps have the highest number of user reviews, this data is heavily skewed by the large number of reviews for Waze and Google Maps. 

For example's sake, let's examine a couple of other categories. 


In [62]:
for i in apple_clean_english_free:
    if i[12] == 'Social Networking':
        print(i[2], '- Total ratings:', i[6])

Facebook - Total ratings: 2974676
LinkedIn - Total ratings: 71856
Skype for iPhone - Total ratings: 373519
Tumblr - Total ratings: 334293
Match™ - #1 Dating App. - Total ratings: 60659
WhatsApp Messenger - Total ratings: 287589
TextNow - Unlimited Text + Calls - Total ratings: 164963
Grindr - Gay and same sex guys chat, meet and date - Total ratings: 23201
imo video calls and chat - Total ratings: 18841
Ameba - Total ratings: 269
Weibo - Total ratings: 7265
Badoo - Meet New People, Chat, Socialize. - Total ratings: 34428
Kik - Total ratings: 260965
Qzone - Total ratings: 1649
Fake-A-Location Free ™ - Total ratings: 354
Tango - Free Video Call, Voice and Chat - Total ratings: 75412
MeetMe - Chat and Meet New People - Total ratings: 97072
SimSimi - Total ratings: 23530
Viber Messenger – Text & Call - Total ratings: 164249
Find My Family, Friends & iPhone - Life360 Locator - Total ratings: 43877
Weibo HD - Total ratings: 16772
POF - Best Dating App for Conversations - Total ratings: 52642

Similar to Navigation apps, we can see that Social Networking apps, too, are skewed by a large number of user reviews for a small amount of apps (e.g. Facebook, Skype & WhatsApp). Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [64]:
for i in apple_clean_english_free:
    if i[12] == 'Reference':
        print(i[2], '- Total ratings:', i[6])

Bible - Total ratings: 985920
Dictionary.com Dictionary & Thesaurus - Total ratings: 200047
Dictionary.com Dictionary & Thesaurus for iPad - Total ratings: 54175
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran - Total ratings: 18418
Merriam-Webster Dictionary - Total ratings: 16849
Google Translate - Total ratings: 26786
Night Sky - Total ratings: 12122
WWDC - Total ratings: 762
Jishokun-Japanese English Dictionary & Translator - Total ratings: 0
教えて!goo - Total ratings: 0
VPN Express - Total ratings: 14
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition - Total ratings: 17588
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools - Total ratings: 4693
Guides for Pokémon GO - Pokemon GO News and Cheats - Total ratings: 826
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free - Total ratings: 718
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) - Total rat

However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now, let's analyse the Google Play market a bit.

# Most Popular Apps by Genre on Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — most values are open-ended (100+, 1,000+, 5,000+, etc.). Nevertheless, we're going to leave the numbers as they are (we don't have any better data to work with), which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [66]:
categories_android = freq_table(android_clean_english_free, 1)

for category in categories_android: 
    total = 0
    len_category = 0
    for app in android_clean_english_free:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)
            
    

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [68]:
for app in android_clean_english_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess


We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [70]:
for app in android_clean_english_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [72]:
for app in android_clean_english_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [75]:
for app in android_clean_english_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.